In [1]:
import os
import csv
import io
import arcpy
import tempfile 
import shutil
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_decimal_from_dms(dms, ref):
    degrees = float(dms[0])
    minutes = float(dms[1]) / 60.0
    seconds = float(dms[2]) / 3600.0
    decimal = degrees + minutes + seconds

    if ref in ['S', 'W']:
        decimal = -decimal

    return decimal

def get_exif_data(image_path):
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
    except Exception as e:
        #print(f"Erreur lors de la lecture de l'image {image_path}: {e}")
        return None
    
    if not exif_data:
        return None
    
    exif = {}
    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        exif[tag_name] = value
    return exif

def get_gps_info(exif_data):
    if "GPSInfo" not in exif_data:
        return None
    
    gps_info = exif_data["GPSInfo"]
    gps_data = {}
    
    for key in gps_info.keys():
        decode = GPSTAGS.get(key, key)
        gps_data[decode] = gps_info[key]
    
    return gps_data

def get_gps_coordinates(image_path):
    exif_data = get_exif_data(image_path)
    
    if not exif_data:
        return None
    
    gps_info = get_gps_info(exif_data)
    
    if not gps_info:
        return None

    gps_latitude = gps_info.get("GPSLatitude")
    gps_latitude_ref = gps_info.get("GPSLatitudeRef")
    gps_longitude = gps_info.get("GPSLongitude")
    gps_longitude_ref = gps_info.get("GPSLongitudeRef")

    if not all([gps_latitude, gps_latitude_ref, gps_longitude, gps_longitude_ref]):
        return None

    lat = get_decimal_from_dms(gps_latitude, gps_latitude_ref)
    lon = get_decimal_from_dms(gps_longitude, gps_longitude_ref)

    return lat, lon

def process_images_in_directory(directory):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.tiff']
    
    # Création d'un fichier CSV temporaire en mémoire
    temp_csv = io.StringIO()
    csv_writer = csv.DictWriter(temp_csv, fieldnames=['latitude', 'longitude'])
    csv_writer.writeheader()  # Écriture des en-têtes du fichier CSV

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(tuple(valid_extensions)):
                image_path = os.path.join(root, file)
                coordinates = get_gps_coordinates(image_path)
                
                if coordinates:
                    csv_writer.writerow({'latitude': coordinates[0], 'longitude': coordinates[1]})
                    #print(f"Image: {image_path} -> Latitude: {coordinates[0]}, Longitude: {coordinates[1]}")
                else:
                    #print(f"Image: {image_path} -> Pas de données GPS trouvées.")
                    pass
    
    # Écrire le CSV en mémoire dans un fichier temporaire sur disque
    temp_csv.seek(0)  # Remettre le curseur au début du fichier
    
    # Créer un fichier temporaire sur disque
    temp_dir = tempfile.mkdtemp()
    csv_file_path = os.path.join(temp_dir, "temp_coordinates.csv")
    
    with open(csv_file_path, 'w', newline='') as f:
        f.write(temp_csv.getvalue())  # Copier le contenu du fichier en mémoire dans le fichier physique
    
    return csv_file_path

def convert_csv_to_points_arcpy(csv_path, output_fc, spatial_ref=4326):
    # Définir l'outil ArcPy Table XY vers Point
    arcpy.env.overwriteOutput = True
    
    # Utilisation de l'outil XY Table to Point
    arcpy.management.XYTableToPoint(
        in_table=csv_path,
        out_feature_class=output_fc,
        x_field="longitude",
        y_field="latitude",
        coordinate_system=arcpy.SpatialReference(spatial_ref)
    )
    
    print(f"Conversion réussie ! Les points ont été sauvegardés dans : {output_fc}")

In [ ]:
# Remplacez dans vos données, en premier le chemin d'accès vers le dossier contenant vos photos, en second la gdb de sortie
folder_path = r"D:\photos_iphone"  # Dossier avec tes images
output_fc = r"D:\Documents\ArcGIS\Projects\hotspot_photos\hotspot_photos.gdb\photoshotspot"  # Sortie Feature Class

# 1. Traiter les images pour extraire les coordonnées GPS
csv_file = process_images_in_directory(folder_path)

# 2. Utiliser le CSV pour créer une Feature Class avec des points
convert_csv_to_points_arcpy(csv_file, output_fc)

# 3. Nettoyer les fichiers temporaires

shutil.rmtree(os.path.dirname(csv_file))  # Supprimer le dossier temporaire